In [1]:
from config import model_name
import pandas as pd
import json
from tqdm import tqdm
from os import path
import random
from nltk.tokenize import word_tokenize
import numpy as np
import csv
from pathlib import Path
from shutil import copyfile
import importlib
import os

In [29]:
source_embedding = pd.read_table('/Users/baeyuna/Documents/SNU_DLab/TANR_Bert/data/glove/glove.840B.300d.txt',
                                     index_col=0,
                                     sep=' ',
                                     header=None,
                                     quoting=csv.QUOTE_NONE,
                                     names=range(300))
source_embedding.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
",",-0.082752,0.672040,-0.14987,-0.064983,0.056491,0.402280,0.002775,-0.331100,-0.306910,2.0817,...,-0.14331,0.018267,-0.18643,0.207090,-0.355980,0.053380,-0.050821,-0.191800,-0.378460,-0.06589
.,0.012001,0.207510,-0.12578,-0.593250,0.125250,0.159750,0.137480,-0.331570,-0.136940,1.7893,...,0.16165,-0.066737,-0.29556,0.022612,-0.281350,0.063500,0.140190,0.138710,-0.360490,-0.03500
the,0.272040,-0.062030,-0.18840,0.023225,-0.018158,0.006719,-0.138770,0.177080,0.177090,2.5882,...,-0.42810,0.168990,0.22511,-0.285570,-0.102800,-0.018168,0.114070,0.130150,-0.183170,0.13230
and,-0.185670,0.066008,-0.25209,-0.117250,0.265130,0.064908,0.122910,-0.093979,0.024321,2.4926,...,-0.59396,-0.097729,0.20072,0.170550,-0.004736,-0.039709,0.324980,-0.023452,0.123020,0.33120
to,0.319240,0.063160,-0.27858,0.261200,0.079248,-0.214620,-0.104950,0.154950,-0.033530,2.4834,...,-0.12977,0.371300,0.18888,-0.004274,-0.106450,-0.258100,-0.044629,0.082745,0.097801,0.25045


In [30]:
source_embedding.index.rename('word', inplace=True)
source_embedding.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
word,,,,,,,,,,,,,,,,,,,,,
",",-0.082752,0.672040,-0.14987,-0.064983,0.056491,0.402280,0.002775,-0.331100,-0.306910,2.0817,...,-0.14331,0.018267,-0.18643,0.207090,-0.355980,0.053380,-0.050821,-0.191800,-0.378460,-0.06589
.,0.012001,0.207510,-0.12578,-0.593250,0.125250,0.159750,0.137480,-0.331570,-0.136940,1.7893,...,0.16165,-0.066737,-0.29556,0.022612,-0.281350,0.063500,0.140190,0.138710,-0.360490,-0.03500
the,0.272040,-0.062030,-0.18840,0.023225,-0.018158,0.006719,-0.138770,0.177080,0.177090,2.5882,...,-0.42810,0.168990,0.22511,-0.285570,-0.102800,-0.018168,0.114070,0.130150,-0.183170,0.13230
and,-0.185670,0.066008,-0.25209,-0.117250,0.265130,0.064908,0.122910,-0.093979,0.024321,2.4926,...,-0.59396,-0.097729,0.20072,0.170550,-0.004736,-0.039709,0.324980,-0.023452,0.123020,0.33120
to,0.319240,0.063160,-0.27858,0.261200,0.079248,-0.214620,-0.104950,0.154950,-0.033530,2.4834,...,-0.12977,0.371300,0.18888,-0.004274,-0.106450,-0.258100,-0.044629,0.082745,0.097801,0.25045


In [31]:
source_embedding.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2196017 entries, , to zulchzulu
Columns: 300 entries, 0 to 299
dtypes: float64(300)
memory usage: 4.9+ GB


In [54]:
word2int = pd.read_table('/Users/baeyuna/Documents/SNU_DLab/NAML/data/train/word2int.tsv', na_filter=False, index_col='word')
word2int

,int
word,
the,1
brands,2
queen,3
elizabeth,4
",",5
...,...
khizr,44771
edwards-helaire,44772
korg,44773


# bert_word_embedding 으로 만들기

In [55]:
word_embeddings = np.load('/Users/baeyuna/Documents/SNU_DLab/TANR_Bert/data/glove/word_embeddings.npy')
word_embeddings

array([[ 0.02291545,  0.01740933,  0.01044573, ...,  0.02647595,
        -0.05486389,  0.001196  ],
       [ 0.02087866,  0.03977818,  0.02973026, ..., -0.04849208,
        -0.03428288, -0.03353313],
       [-0.00754663,  0.00295051, -0.00115429, ..., -0.00839342,
         0.01157734, -0.03211762],
       ...,
       [ 0.03940183, -0.02765681, -0.01243274, ...,  0.01878955,
        -0.01772225,  0.05469738],
       [-0.00503979,  0.01623228,  0.06277147, ..., -0.0290992 ,
        -0.03663587,  0.01084558],
       [-0.01480096,  0.0230887 , -0.04091615, ..., -0.04485734,
        -0.05254643, -0.02245559]], dtype=float32)

In [56]:
word_embeddings.shape

(44775, 768)

In [62]:
df = pd.DataFrame(word_embeddings)
df.loc[:,'words'] = word2int.index.tolist()
df = df.set_index('words')
source_embedding = df.copy()
source_embedding

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
words,,,,,,,,,,,,,,,,,,,,,
the,0.022915,0.017409,0.010446,-0.048069,0.032508,0.013682,-0.043972,0.028114,-0.081688,-0.003506,...,-0.014207,0.003513,-0.056283,0.023020,0.047154,0.024799,0.025471,0.026476,-0.054864,0.001196
brands,0.020879,0.039778,0.029730,-0.002456,0.059781,-0.025582,-0.029728,-0.038067,-0.051656,0.040926,...,-0.004319,0.058174,-0.023548,-0.041602,0.001282,0.009522,0.026286,-0.048492,-0.034283,-0.033533
queen,-0.007547,0.002951,-0.001154,-0.047338,0.037052,0.011151,-0.070805,-0.029634,-0.012547,-0.008933,...,0.030856,0.036562,-0.056421,-0.027629,0.028740,0.003800,0.027521,-0.008393,0.011577,-0.032118
elizabeth,0.019542,-0.045189,0.028748,-0.028304,0.034132,0.049069,-0.076122,0.023633,-0.062191,-0.022393,...,0.014009,0.046649,-0.040938,-0.059904,-0.022907,0.016196,-0.041572,-0.009329,-0.002110,-0.034657
",",-0.021864,0.011111,0.016383,-0.032495,-0.002032,0.032761,-0.019221,-0.006823,-0.076860,-0.039014,...,0.000077,0.017377,-0.059505,0.064820,0.065166,-0.003043,-0.019843,0.010483,-0.066509,-0.006312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
khizr,0.039653,-0.058133,-0.060750,-0.072139,0.026598,0.031439,0.009851,0.040807,-0.045499,-0.059950,...,0.047280,0.013840,-0.034338,0.003750,-0.042403,-0.041079,-0.023069,0.008094,-0.011244,0.047785
edwards-helaire,0.040825,-0.037304,0.005907,-0.023890,-0.007193,-0.037433,-0.071384,-0.007603,-0.018867,0.004944,...,0.046952,0.050125,0.060447,-0.088661,0.059591,0.013882,0.007241,0.041523,-0.038662,0.019831
korg,0.039402,-0.027657,-0.012433,-0.051315,0.026456,-0.026098,-0.015726,0.053842,-0.047304,-0.019760,...,0.005184,0.019469,-0.029712,0.017312,-0.042999,-0.055916,0.010879,0.018790,-0.017722,0.054697


In [63]:
# merged = word2int.merge(source_embedding,
#                             how='inner',
#                             left_index=True,
#                             right_index=True)
# merged.set_index('int', inplace=True)
# merged

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
int,,,,,,,,,,,,,,,,,,,,,
1,0.022915,0.017409,0.010446,-0.048069,0.032508,0.013682,-0.043972,0.028114,-0.081688,-0.003506,...,-0.014207,0.003513,-0.056283,0.023020,0.047154,0.024799,0.025471,0.026476,-0.054864,0.001196
2,0.020879,0.039778,0.029730,-0.002456,0.059781,-0.025582,-0.029728,-0.038067,-0.051656,0.040926,...,-0.004319,0.058174,-0.023548,-0.041602,0.001282,0.009522,0.026286,-0.048492,-0.034283,-0.033533
3,-0.007547,0.002951,-0.001154,-0.047338,0.037052,0.011151,-0.070805,-0.029634,-0.012547,-0.008933,...,0.030856,0.036562,-0.056421,-0.027629,0.028740,0.003800,0.027521,-0.008393,0.011577,-0.032118
4,0.019542,-0.045189,0.028748,-0.028304,0.034132,0.049069,-0.076122,0.023633,-0.062191,-0.022393,...,0.014009,0.046649,-0.040938,-0.059904,-0.022907,0.016196,-0.041572,-0.009329,-0.002110,-0.034657
5,-0.021864,0.011111,0.016383,-0.032495,-0.002032,0.032761,-0.019221,-0.006823,-0.076860,-0.039014,...,0.000077,0.017377,-0.059505,0.064820,0.065166,-0.003043,-0.019843,0.010483,-0.066509,-0.006312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44771,0.039653,-0.058133,-0.060750,-0.072139,0.026598,0.031439,0.009851,0.040807,-0.045499,-0.059950,...,0.047280,0.013840,-0.034338,0.003750,-0.042403,-0.041079,-0.023069,0.008094,-0.011244,0.047785
44772,0.040825,-0.037304,0.005907,-0.023890,-0.007193,-0.037433,-0.071384,-0.007603,-0.018867,0.004944,...,0.046952,0.050125,0.060447,-0.088661,0.059591,0.013882,0.007241,0.041523,-0.038662,0.019831
44773,0.039402,-0.027657,-0.012433,-0.051315,0.026456,-0.026098,-0.015726,0.053842,-0.047304,-0.019760,...,0.005184,0.019469,-0.029712,0.017312,-0.042999,-0.055916,0.010879,0.018790,-0.017722,0.054697


In [64]:
# missed_index = np.setdiff1d(np.arange(len(word2int) + 1),
#                             merged.index.values)
# missed_embedding = pd.DataFrame(data=np.random.normal(
#     size=(len(missed_index), 768)))
# missed_embedding['int'] = missed_index
# missed_embedding.set_index('int', inplace=True)
# missed_embedding

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
int,,,,,,,,,,,,,,,,,,,,,
0,-0.737502,-1.90175,-0.014587,1.887085,2.155873,-0.248314,0.015405,-1.485848,0.121748,1.092142,...,0.526911,-1.82094,0.622896,-0.093258,-0.343962,-0.602187,0.827893,1.403123,-0.80017,0.17592


In [69]:
# final_embedding = pd.concat([merged, missed_embedding]).sort_index()
# final_embedding
np.save('pretrained_word_embedding.npy', merged.values)

In [11]:
import numpy as np

pretrained_word_embedding = np.load('/Users/baeyuna/Documents/SNU_DLab/NAML/data/train/pretrained_word_embedding.npy')
pretrained_word_embedding

array([[-0.78346185,  0.39302635, -0.71049564, ...,  1.1914767 ,
        -0.58732314, -1.08972927],
       [ 0.27204   , -0.06203   , -0.1884    , ...,  0.13015   ,
        -0.18317   ,  0.1323    ],
       [ 0.30791   ,  0.12428   , -0.045487  , ..., -0.4962    ,
         0.56847   ,  0.51508   ],
       ...,
       [ 0.041715  ,  0.85602   ,  0.031382  , ...,  0.75407   ,
        -0.72133   ,  0.19725   ],
       [-0.24537994,  0.24460628, -0.9640435 , ...,  0.70863113,
         0.23182131,  0.37770902],
       [-0.044587  , -0.37936   ,  0.40889   , ...,  0.8848    ,
        -0.41805   , -0.21511   ]])

In [12]:
word_lst = word2int.index.tolist()
len(word_lst)

44775

# Bert WordEmbedding

In [13]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

def get_model(model_url, max_seq_length):
  labse_layer = hub.KerasLayer(model_url, trainable=True)

  # Define input.
  input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                         name="input_word_ids")
  input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                     name="input_mask")
  segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                      name="segment_ids")

  # LaBSE layer.
  pooled_output,  _ = labse_layer([input_word_ids, input_mask, segment_ids])

  # The embedding is l2 normalized.
  pooled_output = tf.keras.layers.Lambda(
      lambda x: tf.nn.l2_normalize(x, axis=1))(pooled_output)

  # Define model.
  return tf.keras.Model(
        inputs=[input_word_ids, input_mask, segment_ids],
        outputs=pooled_output), labse_layer

max_seq_length = 64
labse_model, labse_layer = get_model(
    model_url="https://tfhub.dev/google/LaBSE/1", max_seq_length=max_seq_length)

In [14]:
import bert

vocab_file = labse_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = labse_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file, do_lower_case)



In [15]:
def create_input(input_strings, tokenizer, max_seq_length):

  input_ids_all, input_mask_all, segment_ids_all = [], [], []
  for input_string in input_strings:
    # Tokenize input.
    input_tokens = ["[CLS]"] + tokenizer.tokenize(input_string) + ["[SEP]"]
    input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
    sequence_length = min(len(input_ids), max_seq_length)

    # Padding or truncation.
    if len(input_ids) >= max_seq_length:
      input_ids = input_ids[:max_seq_length]
    else:
      input_ids = input_ids + [0] * (max_seq_length - len(input_ids))

    input_mask = [1] * sequence_length + [0] * (max_seq_length - sequence_length)

    input_ids_all.append(input_ids)
    input_mask_all.append(input_mask)
    segment_ids_all.append([0] * max_seq_length)

  return np.array(input_ids_all), np.array(input_mask_all), np.array(segment_ids_all)

In [16]:
def encode(input_text):
  input_ids, input_mask, segment_ids = create_input(
    input_text, tokenizer, max_seq_length)
  return labse_model([input_ids, input_mask, segment_ids])

In [17]:
english_sentences = ["dog", "Puppies are nice.", "I enjoy taking long walks along the beach with my dog."]

english_embeddings = encode(english_sentences)
english_embeddings

<tf.Tensor: shape=(3, 768), dtype=float32, numpy=
array([[-0.01026385, -0.00157138, -0.01473237, ...,  0.0494036 ,
        -0.01296441, -0.03172865],
       [-0.05280595, -0.00894449, -0.05583495, ..., -0.00754025,
         0.01059245, -0.01610355],
       [ 0.02716724, -0.00592896,  0.01995578, ..., -0.00215263,
         0.00364844, -0.03194168]], dtype=float32)>

In [18]:
np.save('x_save', english_embeddings)

In [23]:
word_lst_embeddings1 = np.load('/Users/baeyuna/Documents/SNU_DLab/TANR_Bert/data/glove/word_lst_embeddings1.npy')
word_lst_embeddings10 = np.load('/Users/baeyuna/Documents/SNU_DLab/TANR_Bert/data/glove/word_lst_embeddings10.npy')


In [34]:
word_lst_embeddings10.shape

(2000, 768)

In [33]:
word_lst_embeddings1.shape

(2000, 768)

In [36]:
np.concatenate([word_lst_embeddings1, word_lst_embeddings10]).shape

(4000, 768)

In [24]:
word_lst_embeddings1.shape

(2000, 768)

In [25]:
word_lst_embeddings1

array([[ 0.02291545,  0.01740933,  0.01044573, ...,  0.02647595,
        -0.05486389,  0.001196  ],
       [ 0.02087866,  0.03977818,  0.02973026, ..., -0.04849208,
        -0.03428288, -0.03353313],
       [-0.00754663,  0.00295051, -0.00115429, ..., -0.00839342,
         0.01157734, -0.03211762],
       ...,
       [ 0.01743446, -0.05396994, -0.0726243 , ...,  0.00198761,
        -0.03869714,  0.01757814],
       [ 0.01157075,  0.03626885, -0.01124088, ...,  0.01655105,
        -0.03771622, -0.04456491],
       [-0.02233236, -0.0285336 , -0.02215027, ...,  0.01747822,
        -0.04677428, -0.01103436]], dtype=float32)

In [21]:
len(word_lst)

44775

In [22]:
44775//2000

22

In [12]:
word_lst_embeddings = encode(word_lst)
word_lst_embeddings

<tf.Tensor: shape=(3, 768), dtype=float32, numpy=
array([[-0.01112244,  0.00602975, -0.00272447, ...,  0.0336603 ,
        -0.02208618, -0.05659664],
       [-0.04087604, -0.01662398, -0.04726062, ...,  0.03640377,
         0.02879217, -0.03352933],
       [-0.01007188, -0.01358392, -0.03919495, ...,  0.0337332 ,
        -0.01138729, -0.05158317]], dtype=float32)>

In [ ]:
np.save('D:/admin/Documents/x_save', word_lst_embeddings)

In [32]:
df = pd.DataFrame(index=range(0,word_lst_embeddings.shape[0]), columns=range(0,word_lst_embeddings.shape[1]))

for i in range(word_lst_embeddings.shape[0]):
    df.iloc[i,:] = word_lst_embeddings[i]

df.loc[:,'words'] = word_lst 
df = df.rename_axis('words').iloc[:,:-1]
print(df.shape)

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.0111224,0.00602975,-0.00272447,0.000836123,-0.0263209,0.0247632,0.0511221,0.0262025,-0.0621353,-0.0558861,...,0.0116968,-0.00990636,0.0337292,-0.014268,-0.0140283,-0.00477358,-0.0121225,0.0336603,-0.0220862,-0.0565966
1,-0.040876,-0.016624,-0.0472606,-0.0340528,-0.0378542,0.0239878,0.0426077,0.00444569,-0.0612404,-0.0501724,...,0.0101625,-0.0207745,0.0282323,-0.0286058,-0.00483788,-0.00685988,0.0179411,0.0364038,0.0287922,-0.0335293
2,-0.0100719,-0.0135839,-0.039195,-0.0589811,0.00538327,0.02524,0.0306217,-0.0189067,-0.0431255,0.0504395,...,-0.0536857,-0.0311351,0.00727072,-0.038046,-0.023034,-0.015535,-0.0318728,0.0337332,-0.0113873,-0.0515832


In [ ]:
merged = word2int.merge(source_embedding,
                            how='inner',
                            left_index=True,
                            right_index=True)
merged.set_index('int', inplace=True)
merged

In [ ]:
missed_index = np.setdiff1d(np.arange(len(word2int) + 1),
                            merged.index.values)
missed_embedding = pd.DataFrame(data=np.random.normal(
    size=(len(missed_index), 300)))
missed_embedding['int'] = missed_index
missed_embedding.set_index('int', inplace=True)
missed_embedding

In [ ]:
final_embedding = pd.concat([merged, missed_embedding]).sort_index()
np.save('pretrained_word_embedding.npy', final_embedding.values)

print(
    f'Rate of word missed in pretrained embedding: {(len(missed_index)-1)/len(word2int):.4f}'
)